In [ ]:
# Импорт необходимых библиотек
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
import os

In [ ]:

def preprocess_image(image_path):
    """
    Предобработка изображения:
    - Коррекция искажений от наклонной камеры
    - Нормализация освещения
    - Фильтрация шумов
    """
    
    # Загрузка изображения
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Не удалось загрузить изображение: {image_path}")
    
    print(f"Оригинальное изображение: {img.shape}")
    
    # 1. Фильтрация шумов
    # Медианный фильтр для удаления соли и перца
    img_median = cv2.medianBlur(img, 5)
    
    # Гауссов фильтр для сглаживания
    img_blur = cv2.GaussianBlur(img_median, (9, 9), 2)
    
    # 2. Нормализация освещения
    # CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_clahe = clahe.apply(img_blur)
    
    # 3. Дополнительная фильтрация
    # Билатеральный фильтр для сохранения границ
    img_bilateral = cv2.bilateralFilter(img_clahe, 9, 75, 75)
    
    return {
        'original': img,
        'median': img_median,
        'gaussian': img_blur,
        'clahe': img_clahe,
        'bilateral': img_bilateral,
        'final': img_bilateral  # финальный результат предобработки
    }

image_path = "Данные/Положение 1/Image__2025-07-15__15-22-22.bmp"
processed_images = preprocess_image(image_path)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(processed_images['original'], cmap='gray')
axes[0, 0].set_title('Оригинальное изображение')
axes[0, 0].axis('off')

axes[0, 1].imshow(processed_images['median'], cmap='gray')
axes[0, 1].set_title('Медианный фильтр')
axes[0, 1].axis('off')

axes[0, 2].imshow(processed_images['gaussian'], cmap='gray')
axes[0, 2].set_title('Гауссов фильтр')
axes[0, 2].axis('off')

axes[1, 0].imshow(processed_images['clahe'], cmap='gray')
axes[1, 0].set_title('CLAHE нормализация')
axes[1, 0].axis('off')

axes[1, 1].imshow(processed_images['bilateral'], cmap='gray')
axes[1, 1].set_title('Билатеральный фильтр')
axes[1, 1].axis('off')

axes[1, 2].imshow(processed_images['final'], cmap='gray')
axes[1, 2].set_title('Финальный результат')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("Предобработка изображения завершена!")

In [ ]:

def detect_reference_points(img_preprocessed):
    """
    Детекция реперных точек (маркеров):
    - Автоматическое обнаружение маркеров на поле
    - Калибровка системы координат
    """
    
    # 1. Детекция кругов (маркеров)
    circles = cv2.HoughCircles(
        img_preprocessed,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=15,
        param1=50,
        param2=15,
        minRadius=5,
        maxRadius=30
    )
    
    if circles is None:
        raise ValueError("Не удалось найти маркеры на изображении")
    
    circles = np.uint16(np.around(circles))
    
    # 2. Сортировка и идентификация маркеров
    centers = []
    for (x, y, r) in circles[0, :]:
        centers.append((x, y, r))
    
    # Сортировка по позиции (top-left, top-right, bottom-left, bottom-right)
    centers.sort(key=lambda p: (p[1], p[0]))  # сначала по y, потом по x
    
    # Разделение на верхние и нижние точки
    top_centers = centers[:2]
    bottom_centers = centers[2:]
    
    # Сортировка верхних и нижних точек по x
    top_centers.sort(key=lambda p: p[0])
    bottom_centers.sort(key=lambda p: p[0])
    
    # Получение четырех угловых точек
    tl = top_centers[0][:2]    # top-left
    tr = top_centers[1][:2]    # top-right
    bl = bottom_centers[0][:2] # bottom-left
    br = bottom_centers[1][:2] # bottom-right
    
    # 3. Калибровка системы координат
    # Вычисление центра поля
    field_center_x = (tl[0] + tr[0] + bl[0] + br[0]) // 4
    field_center_y = (tl[1] + tr[1] + bl[1] + br[1]) // 4
    field_center = (field_center_x, field_center_y)
    
    # Вычисление размеров поля
    field_width = max(
        np.sqrt((tr[0] - tl[0])**2 + (tr[1] - tl[1])**2),
        np.sqrt((br[0] - bl[0])**2 + (br[1] - bl[1])**2)
    )
    field_height = max(
        np.sqrt((bl[0] - tl[0])**2 + (bl[1] - tl[1])**2),
        np.sqrt((br[0] - tr[0])**2 + (br[1] - tr[1])**2)
    )
    
    # 4. Матрица перспективного преобразования
    pts1 = np.float32([tl, tr, bl, br])
    pts2 = np.float32([[0, 0], [2048, 0], [0, 2048], [2048, 2048]])
    transformation_matrix = cv2.getPerspectiveTransform(pts1, pts2)
    
    return {
        'markers': {
            'top_left': tl,
            'top_right': tr,
            'bottom_left': bl,
            'bottom_right': br
        },
        'field_center': field_center,
        'field_dimensions': (field_width, field_height),
        'transformation_matrix': transformation_matrix,
        'all_circles': circles[0]
    }

# Детекция реперных точек
reference_data = detect_reference_points(processed_images['final'])

# Визуализация результатов детекции
img_vis = cv2.cvtColor(processed_images['original'], cv2.COLOR_GRAY2BGR)

# Отображение всех найденных маркеров
for (x, y, r) in reference_data['all_circles']:
    cv2.circle(img_vis, (x, y), r, (0, 255, 0), 2)  # зеленый круг
    cv2.circle(img_vis, (x, y), 2, (0, 0, 255), 3)  # красный центр

# Отображение угловых маркеров
markers = reference_data['markers']
cv2.circle(img_vis, markers['top_left'], 5, (255, 0, 0), -1)      # синий
cv2.circle(img_vis, markers['top_right'], 5, (255, 255, 0), -1)   # голубой
cv2.circle(img_vis, markers['bottom_left'], 5, (255, 0, 255), -1) # пурпурный
cv2.circle(img_vis, markers['bottom_right'], 5, (0, 255, 255), -1) # желтый

# Отображение центра поля
field_center = reference_data['field_center']
cv2.circle(img_vis, field_center, 8, (255, 255, 255), -1)  # белый центр

# Подписи
cv2.putText(img_vis, 'TL', (markers['top_left'][0]+10, markers['top_left'][1]-10), 
           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
cv2.putText(img_vis, 'TR', (markers['top_right'][0]+10, markers['top_right'][1]-10), 
           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
cv2.putText(img_vis, 'BL', (markers['bottom_left'][0]+10, markers['bottom_left'][1]-10), 
           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 255), 2)
cv2.putText(img_vis, 'BR', (markers['bottom_right'][0]+10, markers['bottom_right'][1]-10), 
           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
cv2.putText(img_vis, 'CENTER', (field_center[0]-30, field_center[1]-10), 
           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img_vis, cv2.COLOR_BGR2RGB))
plt.title('Детекция реперных точек и калибровка системы координат')
plt.axis('off')
plt.show()

# Вывод информации о калибровке
print("=== РЕЗУЛЬТАТЫ ДЕТЕКЦИИ РЕПЕРНЫХ ТОЧЕК ===")
print(f"Центр поля: {reference_data['field_center']}")
print(f"Размеры поля: {reference_data['field_dimensions']}")
print(f"Количество найденных маркеров: {len(reference_data['all_circles'])}")
print("\nКоординаты угловых маркеров:")
for name, pos in reference_data['markers'].items():
    print(f"  {name}: {pos}")

In [ ]:

def apply_perspective_correction(img, transformation_matrix, output_size=(2048, 2048)):
    """
    Применение перспективного преобразования для коррекции искажений
    """
    warped = cv2.warpPerspective(img, transformation_matrix, output_size)
    return warped

# Применение перспективного преобразования к оригинальному изображению
warped_original = apply_perspective_correction(
    processed_images['original'], 
    reference_data['transformation_matrix']
)

# Применение к предобработанному изображению
warped_preprocessed = apply_perspective_correction(
    processed_images['final'], 
    reference_data['transformation_matrix']
)

# Визуализация результатов
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(processed_images['original'], cmap='gray')
axes[0].set_title('Оригинальное изображение')
axes[0].axis('off')

axes[1].imshow(warped_original, cmap='gray')
axes[1].set_title('Перспективное преобразование (оригинал)')
axes[1].axis('off')

axes[2].imshow(warped_preprocessed, cmap='gray')
axes[2].set_title('Перспективное преобразование (предобработанное)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("Перспективное преобразование применено!")
print(f"Размер выходного изображения: {warped_preprocessed.shape}")

In [ ]:

def segment_workpiece(img_preprocessed):
    """
    Сегментация заготовки:
    - Выделение границ объекта
    - Определение контуров прямоугольной формы
    """
    
    # 1. Бинаризация изображения
    # Адаптивная пороговая обработка
    binary = cv2.adaptiveThreshold(
        img_preprocessed, 
        255, 
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 
        11, 
        2
    )
    
    # 2. Морфологические операции для очистки
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    
    # 3. Поиск контуров
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 4. Фильтрация контуров по размеру и форме
    workpiece_contours = []
    min_area = 1000  # минимальная площадь контура
    
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area:
            # Аппроксимация контура многоугольником
            epsilon = 0.02 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)
            
            # Проверка на прямоугольную форму (4 угла)
            if len(approx) == 4:
                workpiece_contours.append(approx)
    
    return {
        'binary': binary,
        'contours': contours,
        'workpiece_contours': workpiece_contours,
        'all_contours': contours
    }

# Сегментация заготовки
segmentation_data = segment_workpiece(warped_preprocessed)

# Визуализация результатов сегментации
img_segmentation = cv2.cvtColor(warped_preprocessed, cv2.COLOR_GRAY2BGR)

# Отображение всех контуров (серым)
cv2.drawContours(img_segmentation, segmentation_data['all_contours'], -1, (128, 128, 128), 2)

# Отображение контуров заготовок (зеленым)
cv2.drawContours(img_segmentation, segmentation_data['workpiece_contours'], -1, (0, 255, 0), 3)

# Визуализация
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(warped_preprocessed, cmap='gray')
axes[0].set_title('Предобработанное изображение')
axes[0].axis('off')

axes[1].imshow(segmentation_data['binary'], cmap='gray')
axes[1].set_title('Бинаризованное изображение')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(img_segmentation, cv2.COLOR_BGR2RGB))
axes[2].set_title(f'Сегментация заготовок (найдено: {len(segmentation_data["workpiece_contours"])})')
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("=== РЕЗУЛЬТАТЫ СЕГМЕНТАЦИИ ЗАГОТОВКИ ===")
print(f"Найдено контуров: {len(segmentation_data['all_contours'])}")
print(f"Найдено заготовок: {len(segmentation_data['workpiece_contours'])}")

In [ ]:

def calculate_position(workpiece_contours, transformation_matrix):
    """
    Расчет позиции:
    - Определение геометрического центра
    - Вычисление угла поворота
    - Оценка точности
    """
    
    results = []
    
    for i, contour in enumerate(workpiece_contours):
        # 1. Определение геометрического центра
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            cx, cy = 0, 0
        
        # 2. Вычисление угла поворота
        # Находим минимальный ограничивающий прямоугольник
        rect = cv2.minAreaRect(contour)
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        
        # Угол поворота (в градусах)
        angle = rect[2]
        if rect[1][0] < rect[1][1]:
            angle = angle + 90
        
        # Нормализация угла к диапазону [-90, 90]
        if angle > 90:
            angle = angle - 180
        elif angle < -90:
            angle = angle + 180
        
        # 3. Преобразование координат в систему координат поля
        # Обратное перспективное преобразование
        inv_matrix = cv2.invert(transformation_matrix)[1]
        
        # Преобразуем центр обратно в оригинальные координаты
        center_warped = np.array([[[cx, cy]]], dtype=np.float32)
        center_original = cv2.perspectiveTransform(center_warped, inv_matrix)[0][0]
        
        # 4. Оценка точности
        # Вычисляем площадь и периметр для оценки качества
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        
        # Коэффициент формы (близость к прямоугольнику)
        if perimeter > 0:
            form_factor = 4 * np.pi * area / (perimeter * perimeter)
        else:
            form_factor = 0
        
        # Оценка точности (0-1, где 1 - идеально)
        accuracy = min(1.0, form_factor / 0.8)  # 0.8 - идеальный коэффициент для прямоугольника
        
        results.append({
            'id': i,
            'center_warped': (cx, cy),
            'center_original': (int(center_original[0]), int(center_original[1])),
            'angle': angle,
            'area': area,
            'perimeter': perimeter,
            'form_factor': form_factor,
            'accuracy': accuracy,
            'contour': contour,
            'bounding_box': box
        })
    
    return results

# Расчет позиций заготовок
position_data = calculate_position(
    segmentation_data['workpiece_contours'], 
    reference_data['transformation_matrix']
)

img_position = cv2.cvtColor(warped_preprocessed, cv2.COLOR_GRAY2BGR)

for result in position_data:
    cv2.drawContours(img_position, [result['contour']], -1, (0, 255, 0), 2)
    
    cv2.drawContours(img_position, [result['bounding_box']], -1, (255, 0, 0), 2)
    
    # Отображение центра
    center = result['center_warped']
    cv2.circle(img_position, center, 5, (0, 0, 255), -1)
    
    # Подписи с информацией
    text = f"ID:{result['id']} A:{result['angle']:.1f}°"
    cv2.putText(img_position, text, (center[0]+10, center[1]-10), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)

plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(img_position, cv2.COLOR_BGR2RGB))
plt.title('Расчет позиции заготовок')
plt.axis('off')
plt.show()

# Вывод результатов
print("=== РЕЗУЛЬТАТЫ РАСЧЕТА ПОЗИЦИИ ===")
for result in position_data:
    print(f"\nЗаготовка {result['id']}:")
    print(f"  Центр (преобразованные координаты): {result['center_warped']}")
    print(f"  Центр (оригинальные координаты): {result['center_original']}")
    print(f"  Угол поворота: {result['angle']:.1f}°")
    print(f"  Площадь: {result['area']:.0f} пикселей")
    print(f"  Точность: {result['accuracy']:.2f}")

In [ ]:

os.makedirs('results', exist_ok=True)

cv2.imwrite('results/preprocessed_image.png', processed_images['final'])

cv2.imwrite('results/detected_markers.png', img_vis)

cv2.imwrite('results/perspective_corrected.png', warped_preprocessed)

cv2.imwrite('results/segmentation_result.png', img_segmentation)
cv2.imwrite('results/position_result.png', img_position)

calibration_data = {
    'field_center': reference_data['field_center'],
    'field_dimensions': reference_data['field_dimensions'],
    'markers': {k: list(v) for k, v in reference_data['markers'].items()},
    'transformation_matrix': reference_data['transformation_matrix'].tolist()
}

workpiece_data = []
for result in position_data:
    workpiece_data.append({
        'id': result['id'],
        'center_warped': result['center_warped'],
        'center_original': result['center_original'],
        'angle': result['angle'],
        'area': result['area'],
        'accuracy': result['accuracy']
    })

final_results = {
    'field_calibration': calibration_data,
    'workpieces': workpiece_data,
    'summary': {
        'total_workpieces': len(workpiece_data),
        'average_accuracy': np.mean([w['accuracy'] for w in workpiece_data]) if workpiece_data else 0,
        'processing_time': 'completed'
    }
}

with open('results/calibration_data.json', 'w') as f:
    json.dump(calibration_data, f, indent=2)

with open('results/final_results.json', 'w') as f:
    json.dump(final_results, f, indent=2)


print(f"\nНайдено заготовок: {len(workpiece_data)}")
if workpiece_data:
    print(f"Средняя точность: {np.mean([w['accuracy'] for w in workpiece_data]):.2f}")
    print(f"Средний угол поворота: {np.mean([w['angle'] for w in workpiece_data]):.1f}°")

print("1. results/preprocessed_image.png")
print("2. results/detected_markers.png")
print("3. results/perspective_corrected.png")
print("4. results/segmentation_result.png")
print("5. results/position_result.png")
print("6. results/calibration_data.json")
print("7. results/final_results.json")